In [1]:
# Changing the the directory where all modules are saved.
import os
os.chdir(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Object detection\object detection\object detection\utils')

In [2]:
# Importing all needed libraries.
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision

from engine import train_one_epoch, evaluate
import utils
import transforms as T
from PIL.ExifTags import TAGS, GPSTAGS

c:\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
def get_model(num_classes):
    '''
        This function is used to create a pretrained FastRCNN Model.
    :num_classes: int
        The number of classes. Should be setted n+1 wher n is the number of object to detect.
    '''
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    #get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

In [4]:
import math
def blur(image, coords ,chunk_size=5):
    x1, x2, y1, y2 = coords
    image = image[x1:x2, y1:y2]
    y, x, c = image.shape
    x_steps = math.ceil(x / chunk_size)
    y_steps = math.ceil(y / chunk_size)
    
    
    for i in range(y_steps):
        for j in range(x_steps):
            x_start = j * chunk_size
            x_end = min(x - 1, (j + 1) * chunk_size)
            y_start = i * chunk_size
            y_end = min(y - 1, (i + 1) * chunk_size)
            
            b, g, r = [int(x) for x in cv2.mean(image[y_start:y_end, x_start:x_end, :])[:3]]
            cv2.rectangle(image, (x_start, y_start), (x_end, y_end), (b, g, r), -1)

In [5]:
import numpy as np
import cv2
from PIL import ImageDraw,ImageFilter


device = torch.device('cuda')
loaded_model = get_model(num_classes=2)
loaded_model.load_state_dict(torch.load(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\logo_detection_model\fix1_ports_model.pth'))
loaded_model.to(device)
loaded_model.eval()
cap = cv2.VideoCapture(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\video_test\video.mp4')
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\video_test\project2.avi',cv2.VideoWriter_fourcc(*"MJPG"), fps, (int(cap.get(3)),int(cap.get(4))))


transform = T.Compose([T.ToTensor()])
while cap.isOpened():
    ret, frame = cap.read()
    
    if ret==False:
        break
    img = Image.fromarray(frame).convert('RGB')

    frame = transform(img,'0')

    img, _ = frame
    # Making the prediction.
    with torch.no_grad():
        prediction = loaded_model([img.to(device)])

    # Getting an drawing the image.
    image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)

    for element in range(len(prediction[0]['boxes'])):
        boxes = prediction[0]['boxes'][element].cpu().numpy()
        score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals=4)

        if score > 0.3:
            draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])],
                         outline='red', width=3)
            print(boxes[0])
            print(type(boxes[0]))
            box = (int(boxes[0]),int(boxes[1]),int(boxes[2]),int(boxes[3]))
            ic = image.crop(box)
            for i in range(10):  # with the BLUR filter, you can blur a few times to get the effect you're seeking
                ic = ic.filter(ImageFilter.BLUR)
            image.paste(ic, box)

            #image[] = original_image.filter(ImageFilter.BLUR)
            #blur(image,(int(boxes[0]), int(boxes[2]),int(boxes[1]), int(boxes[3])))
            draw.text((boxes[0], boxes[1]), text=str(score))

    frame = np.array(image)

#blur(frame,coords)

    out.write(frame)
    #cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
    
out.release()
cap.release()
cv2.destroyAllWindows()

c:\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 

In [ ]:
# def get_transform(train = False):
#     '''
#         Uses to create a transformer.
#     :param train: bool
#         Decides if to add or not the RandomHorizontalFlip transformation.
#     '''
#     transforms = []
#     # convert the image, a PIL image, into a PyTorch Tensor
#     transforms.append(T.ToTensor())
#     if train:
#         # during the training, randomly flip the trinaing images
#         # and ground-truth for data augmentation
#         transforms.append(T.RandomHorizontalFlip(0.5))
#     return T.Compose(transforms)

In [ ]:
# import numpy as np
# import cv2
# cap = cv2.VideoCapture(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\video_test\Times Square - Manhattan, New York [HQ].mp4')
# print(cap)
# while cap.isOpened():
#     ret, frame = cap.read()
    
#     Image.fromarray(img)
#     cv2.imshow('frame',frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [15]:
# import numpy as np
# import cv2
# from PIL import ImageDraw

# loaded_model = get_model(num_classes=2)
# loaded_model.load_state_dict(torch.load(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\logo_detection_model\fix1_ports_model.pth'))

# cap = cv2.VideoCapture(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\video_test\Times Square - Manhattan, New York [HQ].mp4')
# print(cap)
# while cap.isOpened():
#     ret, frame = cap.read()
    
#     img = Image.fromarray(frame).convert('RGB')

#     frame = T.Compose([T.ToTensor()])(img,'0')
    
#     img, _ = frame
    
#     loaded_model.eval()
#     # Making the prediction.
#     with torch.no_grad():
#         prediction = loaded_model([img])

#     # Getting an drawing the image.
#     image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
#     draw = ImageDraw.Draw(image)

#     # Drawing the real box around the object.
# #     for elem in range(len(label_boxes)):
# #         draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
# #                        (label_boxes[elem][2], label_boxes[elem][3])],
# #                       outline='green', width=3)
#     # Drawing the predicted box around the object.
#     for element in range(len(prediction[0]['boxes'])):
#         boxes = prediction[0]['boxes'][element].cpu().numpy()
#         score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals=4)

#         if score > 0.3:
#             draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])],
#                          outline='red', width=3)
#             draw.text((boxes[0], boxes[1]), text=str(score))
    

#     frame = np.array(image)
#     cv2.imshow('frame',frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
size

In [ ]:

for filename in glob.glob('C:/New folder/Images/*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
os.chdir(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\images_test')
path_dir = os.getcwd()
paths = os.listdir()
#paths = [r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\images_test\900px-Times_Square_Panorama.jpg',r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\images_test\newFile-3.jpg',r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\images_test\TimesSquare_bright.jpg']
for path in paths:
    img_path = os.path.join(path_dir,path)
    img = Image.open(img_path).convert('RGB')
    # Readding the image metadata.
    for orientation in TAGS.keys() : 
        if TAGS[orientation]=='Orientation' : break 
    # Getting the exif
    exif=dict(img.getexif().items())
    # Rotating the image if the orientation is wrong.
    #         print(img)
    #         print(img.getexif())
    #print(TAGS.keys())
    if len(exif)!=0:
        if orientation in exif.keys():
            if   exif[orientation] == 3 : 
                img=img.rotate(180, expand=True)
            elif exif[orientation] == 6 : 
                img=img.rotate(270, expand=True)
            elif exif[orientation] == 8 : 
                img=img.rotate(90, expand=True)
                
    img = T.Compose([T.ToTensor()])(img,'0')
    imgs.append(img)

NameError: name 'imgs' is not defined

In [18]:
loaded_model = get_model(num_classes=2)
loaded_model.load_state_dict(torch.load(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\logo_detection_model\fix1_ports_model.pth'))

<All keys matched successfully>

In [1]:
from PIL import ImageDraw
# Getting the image.
for idx in range(len(imgs)):
    img, _ = imgs[idx]
    # Getting the object coordinates.
#     label_boxes = np.array(test[idx][1]['boxes'])

    # Setting the model to eval state.
    loaded_model.eval()
    # Making the prediction.
    with torch.no_grad():
        prediction = loaded_model([img])

    # Getting an drawing the image.
    image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)

    # Drawing the real box around the object.
#     for elem in range(len(label_boxes)):
#         draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
#                        (label_boxes[elem][2], label_boxes[elem][3])],
#                       outline='green', width=3)
    # Drawing the predicted box around the object.
    for element in range(len(prediction[0]['boxes'])):
        boxes = prediction[0]['boxes'][element].cpu().numpy()
        score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals=4)

        if score > 0.3:
            draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])],
                         outline='red', width=3)
            draw.text((boxes[0], boxes[1]), text=str(score))
    display(image)